# Langchain agent with tools (duckduckgo, memory)

In [1]:
import sys
sys.path.append("..") # Adds higher directory to python module path

In [2]:
OPENAI_API_KEY =  "" #my open AI key


## DuckduckGO is so shit - do not use it

In [15]:
# https://www.youtube.com/watch?v=biS8G8x8DdA&list=PL8motc6AQftk1Bs42EW45kwYbyJ4jOdiZ&index=20
# https://colab.research.google.com/drive/16gWpUMOfRsvXDVPGtNwwDLV49SyxkJaD?usp=sharing
#https://python.langchain.com/docs/integrations/tools/ddg
from langchain.tools import DuckDuckGoSearchRun

In [23]:
from langchain import OpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
llm = OpenAI(temperature=0,model_name="text-davinci-003", openai_api_key=OPENAI_API_KEY)


In [6]:
from langchain.agents import Tool
from langchain.tools import DuckDuckGoSearchRun

In [7]:
search = DuckDuckGoSearchRun()
#search.run("when is the application deadline for TUM informatics winster semester?")

In [8]:
# describe how your tool will be used in the chain
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run, # this is how the function be called
    description="Useful when you need to do a search on the internet to find information that another tool can't find. be specific with your input"
)

In [9]:
# tools just a list
tools = []
tools.append(duckduckgo_tool)

## Using the agents

In [10]:
from langchain.agents import initialize_agent

In [24]:
zero_shot_agent = initialize_agent(
    agent='zero-shot-react-description', # give tyep of agent - react!
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3 # max iteration is important to save your money...
)

In [16]:
# check that... the agnet it self it just a prompt engineering result
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

DuckDuckGo Search: Useful when you need to do a search on the internet to find information that another tool can't find. be specific with your input

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [DuckDuckGo Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [13]:
# modify the defualt prompt
customer_template = '''

Answer the following questions as best you can. 

You have access to the following tools:\n\nDuckDuckGo Search: Useful when you need to do a search on the internet to find information that another tool can't find. be specific with your input\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [DuckDuckGo Search]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}

'''

In [17]:
zero_shot_agent.agent.llm_chain.prompt.template

"Answer the following questions as best you can. You have access to the following tools:\n\nDuckDuckGo Search: Useful when you need to do a search on the internet to find information that another tool can't find. be specific with your input\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [DuckDuckGo Search]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}"

## Access the tools

In [19]:
general_program_info_template = """
Introduction about {program_name} at {university_name}
"""

In [20]:
formatted_general_program_intro = general_program_info_template.format(program_name="Informatics", university_name="TUM")

In [21]:
formatted_general_program_intro

'\nIntroduction about Informatics at TUM\n'

In [25]:
zero_shot_agent.run(formatted_general_program_intro)



> Entering new AgentExecutor chain...
 Researching the topic
Action: DuckDuckGo Search
Action Input: "Informatics TUM"
Observation: Informatics is a modern discipline and the engine of innovation for technical progress in all fields of economic and social life. Specialists in informatics shape the future! ... In this procedure, the TUM school or department determines whether you meet the specific requirements for its master's degree program. In the initial stages, the ... Integrated research and degree programs designed for the future: The TUM School of Computation, Information and Technology (CIT) at the Technical University of Munich (TUM) unites the disciplines of Mathematics, Informatics, and Electrical and Computer Engineering. TUM School of Computation, Information and Technology - TUM School of Computation, Information and Technology Wir sind die TUM School of Computation, Information and Technology der Technischen Universität München! Alles über unsere Lehre und Forschung. In

'The TUM School of Computation, Information and Technology (CIT) at the Technical University of Munich (TUM) unites the disciplines of Mathematics, Informatics, and Electrical and Computer Engineering. It offers integrated research and degree programs designed for the future.'

In [26]:
# Obervation: the agent re-format my query into duckduckgo, here i might need the prompt guidance!
zero_shot_agent.run("When is the application deadline for the TUM informatics on winter semester")



> Entering new AgentExecutor chain...
 I need to find out when the application deadline is
Action: DuckDuckGo Search
Action Input: "TUM informatics application deadline winter semester"
Observation: Download. ICS, 1 MB. Semester Structure. Summer semester 2023: 01.04.2023 - 30.09.2023. Winter semester 2023/24: 01.10.2023 - 31.03.2024. Summer semester 2024: 01.04.2024 - 30.09.2024. Winter semester 2024/25: 01.10.2024 - 31.03.2025. Periods and Deadlines. Application for Bachelor's. Application for Master's. You must hand in your plan at the latest by the end of November in the wintersemester and by the end of May in the summersemester. Depending on your major, you can select your subjects in advance before the program starts by using our study structure chart. Welcome to TUM Informatics (Computer Science)! As a student at one of TUM CIT Informatics's international partner universities, you can spend a semester or academic year on exchange in the area of Computer Science and Informatics

'The application deadline for the TUM informatics on winter semester is the end of November.'

In [29]:
zero_shot_agent.run("please list out all the required documents e.g. CV, motivation letter, essay, etc for apply master degree in Informatics at Technical university of Munich ")



> Entering new AgentExecutor chain...
 I need to find out what documents are required
Action: DuckDuckGo Search
Action Input: "required documents for master degree in Informatics at Technical university of Munich"
Observation: City. Technical University of Munich. Informatics. M.Sc. (Master of Science) Subjects: Computer Science, Informatics. M.Sc. Degree. 120. ECTS. English & German. Language. Winter semester: 31 May. Summer semester: 30 Nov. Application Deadlines. 2 Years (starting: Both semesters) Duration. Free. Tuition Fees (per semester) Starting. Both semesters. Here you will find all the legal foundations for your studies at TUM: from the General Academic and Examination Regulations via the Program-Specific Academic and Examination Regulations to the Regulations on the Awarding of Doctoral Degrees - official versions as well as engrossed versions. Explanations of the Legal Framework. In the future, master's students from both universities will be able to obtain a master's deg

'The required documents for applying for a master degree in Informatics at Technical University of Munich are: 1) Online Application, 2) Passport Copy, 3) Official High School Transcript, 4) Application fee of 75.00 EUR.'

## React agent with search and vecotr stores as tools!
* https://python.langchain.com/docs/modules/memory/agent_with_memory (here the memory means the chat memory)
* https://python.langchain.com/docs/modules/agents/how_to/agent_vectorstore

In [3]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import LLMChain
#from langchain.utilities import GoogleSearchAPIWrapper this is in general too expensive
from langchain.tools import DuckDuckGoSearchRun

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY) # text-embedding-ada-002

persist_directory = 'vectorDatabase'

## Here is the new embedding being used
embedding = embeddings

# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

retriever = vectordb.as_retriever(search_type='similarity', search_kwargs={"k":3}) # force only return 3 results

In [7]:
from langchain.chains import RetrievalQA 

# llm
llm = OpenAI(temperature=0,model_name="text-davinci-003", openai_api_key=OPENAI_API_KEY)


# create a retrieval chain
program_info_chain = RetrievalQA.from_chain_type(
                                            llm=llm, 
                                            chain_type="stuff", 
                                            retriever=retriever,
                                            return_source_documents=False)

In [12]:
# Tool chain 

search = DuckDuckGoSearchRun()
tools =[
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to asnwer questions about current events"
    ),
    Tool(
        name="Program information QA system",
        func=program_info_chain.run, 
        description="userful for when you need any information about the program infroamtion"
    )
]

In [13]:
# create the agent
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent='zero-shot-react-description', # give tyep of agent - react!
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3 # max iteration is important to save your money...
)

In [14]:
zero_shot_agent.run("What are the required documents for apply TUM informatics e.g. CV, motivaition letter, Essay?")



> Entering new AgentExecutor chain...
 I need to find out what documents are required for the application process.
Action: Program information QA system
Action Input: What documents are required for the TUM informatics application?
Observation:  You will need to submit personal data and information about your education, as well as any required documents. Prospective students with a Bachelor's degree from outside the European Union will also need to request a preliminary examination documentation from uni-assist (VPD). If you are admitted, you will need to submit additional documents as certified copies for the enrollment.
Thought: I now know the final answer
Final Answer: The required documents for the TUM informatics application are personal data and information about your education, as well as any required documents. Prospective students with a Bachelor's degree from outside the European Union will also need to request a preliminary examination documentation from uni-assist (VPD). 

"The required documents for the TUM informatics application are personal data and information about your education, as well as any required documents. Prospective students with a Bachelor's degree from outside the European Union will also need to request a preliminary examination documentation from uni-assist (VPD). If you are admitted, you will need to submit additional documents as certified copies for the enrollment."

# Custom functions with OpenAI functions Agents
* [Youtube](https://www.youtube.com/watch?v=4KXK6c6TVXQ&list=PL8motc6AQftk1Bs42EW45kwYbyJ4jOdiZ&index=30)
    * [colab](https://colab.research.google.com/drive/1o5N38guLVSLHWQvLGUTW8uxjvLeAOhx9?usp=sharing)
* [LangChain doc](https://python.langchain.com/docs/modules/agents/how_to/custom-functions-with-openai-functions-agent)